In [1]:
### Investment data
#Real investment kucoin investment data 
#Initialize investments.ipynb file in Resources folder.
# Prepare API calls for investment data collection.
# Pull investment data from first exchange.
# Organize investment data.
# Pull investment data from remaining exchanges.
# Organize investment data from remaining exchanges

In [2]:
## Defining functions for investment data in Kucoin exchange

# get_headers(url): returns the headers needed for API authentication and authorization.
# get_account_data(url='/api/v1/accounts'): retrieves account data for a user.
# get_investment_data(url='/api/v1/accounts'): retrieves investment data for a user, specifically trade balances.
# get_all_tickers(url='/api/v1/symbols'): retrieves a list of all available trading pairs on the exchange.
# get_tickers(url='/api/v1/market/allTickers'): retrieves the ticker data for all trading pairs on the exchange.
# get_all_tickers: retrieves a list of all available trading pairs (tickers) on KuCoin.
# get_tickers: retrieves ticker information for all trading pairs on KuCoin.
# get_open_orders: retrieves a list of all currently open orders for a given trading pair on KuCoin.
# get_order_history: retrieves a list of all completed orders for a given trading pair on KuCoin.
# get_trade_history: retrieves a list of all trades (fills) for a given trading pair on KuCoin.
# place_order: places a limit order for a given trading pair on KuCoin.
# cancel_order: cancels an order with the given ID for a given trading pair on KuCoin.
# get_balance: retrieves the balance of a given currency in the user's KuCoin account.
# get_all_balances: retrieves the balance of all currencies in the user's KuCoin account.
# get_investment_data: retrieves investment data for the user, including currency, balance, and purchase price
# get_all_tickers: retrieves ticker data for all available currencies
# get_total_profit_loss: calculates the total profit or loss for the user's investments
# get_currency_holdings: retrieves the user's holdings for a specific currency
# get_exchange_balances: calculates the user's balances in USD for all currencies available on the exchange
# get_latest_price: retrieves the latest price for a specific currency
# get_price_history: retrieves the price history for a specific currency over a specified time period

In [3]:
import base64
import hmac
import json
import time

import hashlib
import requests
import pandas as pd
from dotenv import find_dotenv, load_dotenv
import os
from os import environ as env

In [4]:
# Load .env file
ENV_FILE = find_dotenv()
if ENV_FILE:
    load_dotenv(ENV_FILE)
    
api_key = env.get('BITGET_API_KEY')
api_passphrase = env.get('BITGET_API_PASSPHRASE')
api_secret = env.get('BITGET_API_SECRET')

#### Get timestamp

In [5]:
def get_timestamp():
    return int(time.time() * 1000)

#### Set signature

In [6]:
def sign(message, secret_key):
    mac = hmac.new(bytes(secret_key, encoding='utf-8'), bytes(message, encoding='utf-8'), digestmod='sha256')
    d = mac.digest()
    return base64.b64encode(d)

#### Set pre-hash

In [7]:
def pre_hash(timestamp, method, request_path, body):
    return str(timestamp) + str.upper(method) + request_path + body

#### Parse parameters to string

In [8]:
def parse_params_to_str(params):
    url = '?'
    for key, value in params.items():
        url = url + str(key) + '=' + str(value) + '&'

    return url[0:-1]

#### Get headers

In [9]:
def get_headers(api_key, api_passphrase, api_secret, body, method, params, request_path):
    timestamp = str(get_timestamp())
    content = timestamp + method + request_path + parse_params_to_str(params)
    signature = sign(content, api_secret)
    headers = {
        "Content-Type": 'application/json',
        "ACCESS-KEY": api_key,
        "ACCESS-SIGN": signature,
        "ACCESS-TIMESTAMP": timestamp,
        "ACCESS-PASSPHRASE": api_passphrase,
        "locale": 'en-US',
    }
    return headers

#### GET accounts

In [10]:
def get_accounts(api_key, api_passphrase, api_secret):
    body = ""
    method = 'GET'
    params = {'productType':'UMCBL'}
    request_path = '/api/mix/v1/account/accounts'
    header = get_headers(api_key, api_passphrase, api_secret, body, method, params, request_path)
    url = 'https://api.bitget.com' + request_path + parse_params_to_str(params)
    response = requests.get(url, headers=header)
    if response.status_code == 200:
        balances = response.json()
        return balances
    else:
        print('Failed to get accounts. Error code:', response.status_code)
        return None

get_accounts(api_key, api_passphrase, api_secret)

{'code': '00000',
 'msg': 'success',
 'requestTime': 1686325752239,
 'data': [{'marginCoin': 'USDT',
   'locked': '0',
   'available': '1200.77892388',
   'crossMaxAvailable': '153.97275608',
   'fixedMaxAvailable': '153.97275608',
   'maxTransferOut': '153.97275608',
   'equity': '402.66964388',
   'usdtEquity': '402.669643884783',
   'btcEquity': '0.0151811857',
   'crossRiskRate': '0.160390382243',
   'unrealizedPL': '-798.10928000401',
   'bonus': '0'}]}